# Petrol Consumption: Multiple Linear Regression
## Intro
Multiple linear regression is similar to a simple linear regression except you are looking at the relationship between multiple independent variables this time. The main differences for multiple (vs. simple) regression include:
* Taking into account different scales/distributions for independent variables
* Determining the differing effects of each variable on the dependent variable

**Project Link:** https://stackabuse.com/linear-regression-in-python-with-scikit-learn/

## Initial Investigation
To begin with, let's take a look at the data.

In [126]:
# load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# show plots in notebook
%matplotlib inline

# load data
car_df = pd.read_csv('petrol_consumption.csv')

# remove irrelevant variables
car_df.drop(['Index', 'Ones'], axis=1, inplace=True)

# peek at data
car_df.head()

,Petrol tax (cents per gallon),Average income (dollars),Paved Highways (miles),Proportion of population with driver's licenses,Consumption of petrol (millions of gallons)
0,9.0,3571,1976,0.525,541
1,9.0,4092,1250,0.572,524
2,9.0,3865,1586,0.580,561
3,7.5,4870,2351,0.529,414
4,8.0,4399,431,0.544,410


In [23]:
car_df.describe()

,Petrol tax (cents per gallon),Average income (dollars),Paved Highways (miles),Proportion of population with driver's licenses,Consumption of petrol (millions of gallons)
count,48.000000,48.000000,48.000000,48.000000,48.000000
mean,7.668333,4241.833333,5565.416667,0.570333,576.770833
std,0.950770,573.623768,3491.507166,0.055470,111.885816
min,5.000000,3063.000000,431.000000,0.451000,344.000000
25%,7.000000,3739.000000,3110.250000,0.529750,509.500000
50%,7.500000,4298.000000,4735.500000,0.564500,568.500000
75%,8.125000,4578.750000,7156.000000,0.595250,632.750000
max,10.000000,5342.000000,17782.000000,0.724000,968.000000


We can see that there are 4 independent variables (petrol tax, average income, paved highways and driver's license proportion) which potentially affect our dependent variable (petrol consumption).

It's clear that the 4 variables are significantly different in terms of variance and mean, therefore feature scaling and standardization is a good initial step so that our algorithm can fairly process all 4 variables without giving preference to some over others based on sheer numerical size of values.

## Feature Standardization
It is important to split your data into the train and test sets **before** applying scaling/normalization/standardization, otherwise you will shift your data around a mean and standard deviation which will no longer be true once you split your datasets.

Here we will use standardization to scale (standard deviation of 1) and adjust the mean (set to 0) for all 4 variables so that they are all comparable and fit smoothly into our model. Notice that you can use a **pipeline** in sklearn to chain together transformations of your data.

It is also worth noting that as well as standardizing our features, we will also scale and adjust our label/target variable as well. This ensures that there are no issues with weighting or confusion over value ranges within our model.

Further notes on **feature scaling**: https://machinelearningmastery.com/how-to-transform-target-variables-for-regression-with-scikit-learn/

In [121]:
# split into X and y
X = car_df.iloc[:,:4].values
y = car_df.iloc[:,4].values

# load libraries
from sklearn.model_selection import train_test_split

# split data into train/test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# reshape variables to fit model
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# load libraries
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# prepare the model and scale inputs/features
pipeline = Pipeline(steps=[('standardize', preprocessing.StandardScaler()), ('model', LinearRegression())])

# fit pipeline to training data
pipeline.fit(X_train, y_train)

# make predictions
y_pred = pipeline.predict(X_test) # passes test data through both standardization and linear model (due to pipeline)

# show results
y_pred

array([[469.39198872],
       [545.64546431],
       [589.66839402],
       [569.7304133 ],
       [649.77480909],
       [646.63116356],
       [511.60814841],
       [672.47517717],
       [502.07478157],
       [501.2707342 ]])

**NOTE:** the above code applied scaling to the features only, however if we want to scale the target variable (y) also, we can use the below code to do so. This is again useful if the scale of the target variable is extreme in comparison with the feature variables and we want to remove all chance of this throwing off our results.

The first block of code below is quite a manual way of scaling the target variable, it requires us to:
* Create a scaler for the target variable
* Fit this to our training target data
* Apply this scaling to both train and test target data
* Make predictions of our targets using our existing model
* Invert the target values using our target scaler to ensure the results are the same scale as the original target values

In [123]:
# target scaler
y_scaler = preprocessing.StandardScaler() # create object
y_scaler.fit(y_train) # fit to training targets

# transform target variables using model built on train data
y_train = y_scaler.transform(y_train) # scale train
y_test = y_scaler.transform(y_test) # scale test

# predict and invert
y_pred = pipeline.predict(X_test) # make predictions
y_pred = y_scaler.inverse_transform(y_pred) # invert back to original scales for use as actual values

# show results
y_pred

array([[469.39198872],
       [545.64546431],
       [589.66839402],
       [569.7304133 ],
       [649.77480909],
       [646.63116356],
       [511.60814841],
       [672.47517717],
       [502.07478157],
       [501.2707342 ]])

As mentioned, the above code is quite manual, it also means that we are unable (or less able) to use certain convenience functions such as *cross_val_score*. Therefore, we will use a more automated method to achieve the same result below.

Here, we build our pipeline once again and then create a model which uses *TransformedTargetRegressor* to scale our target variable whilst incorporating the scaling and fitting of our training data using our existing pipeline. Therefore it does essentially the same as the more manual process above, except the model object it creates can be used with other functions such as cross_val_score (see below).

In [129]:
# load libraries
from sklearn.compose import TransformedTargetRegressor
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# split into X and y
X = car_df.iloc[:,:4].values
y = car_df.iloc[:,4].values

# load libraries
from sklearn.model_selection import train_test_split

# split data into train/test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# reshape variables to fit model
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# prepare the model and scale inputs/features
pipeline = Pipeline(steps=[('standardize', preprocessing.StandardScaler()), ('model', LinearRegression())])

# define the target transform wrapper
wrapped_model = TransformedTargetRegressor(regressor=pipeline, transformer=preprocessing.StandardScaler())

# use the target transform wrapper
wrapped_model.fit(X_train, y_train)
y_pred = wrapped_model.predict(X_test)

# show results
y_pred

array([[469.39198872],
       [545.64546431],
       [589.66839402],
       [569.7304133 ],
       [649.77480909],
       [646.63116356],
       [511.60814841],
       [672.47517717],
       [502.07478157],
       [501.2707342 ]])

## Cross-Validation of Results
Finally, we can assess the accuracy of our model and cross-validate the results. The final MAE returned is 52.6 which is a bit better than if we'd have not applied scaling of features and labels (see bottom of "A Basic Solution" below for MAE of 56.8). This score improvement would be significantly higher if our dataset were larger as well.

Further notes on **K-Folds Cross-Validation**: https://machinelearningmastery.com/k-fold-cross-validation/

In [137]:
# load libraries
from sklearn.model_selection import KFold, cross_val_score

# evaluate model
cv = KFold(n_splits=10, shuffle=True, random_state=1) # k-folds cross validation
scores = cross_val_score(wrapped_model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1) # calculate scores

# convert scores to positive
scores = abs(scores)

# summarize the result
s_mean = np.mean(scores)
print('Mean MAE: %.3f' % (s_mean))

Mean MAE: 52.588


# A Basic Solution
**NOTE:** the above dives into the multiple linear regression in a little more detail, you can instead run the below code if you want to quickly process a multiple linear regression without scaling and cross-validation.

## Multiple Linear Regression
As in a simple linear regression, we can simply fit our regression model to the data we have.

The model will find the optimum co-efficients for each variable and we will print these out below in order to understand each independent variable's impact on the dependent variable.

In [16]:
# load libraries
from sklearn.linear_model import LinearRegression

# instantiate model object
regr = LinearRegression()

# fit model to training data
regr.fit(X_train, y_train)

# show independent variable's co-efficients
coeff_df = pd.DataFrame(regr.coef_, X.columns, columns=['Coefficient'])
coeff_df

,Coefficient
Petrol tax (cents per gallon),-40.016660
Average income (dollars),-0.065413
Paved Highways (miles),-0.004741
Proportion of population with driver's licenses,1341.862121


The above table tells us the following:
* Changing average income or length of paved highways has almost no effect on petrol consumption
* For every unit increase in petrol tax, petrol consumption will decrease by 40 million gallons
* For every unit inrease in % of people with a driver's license, petrol consumption will increase by 1342 million gallons of petrol

## Model Accuracy

Let's assess the accuracy of the model we've built.

In [17]:
# make predictions using test data
y_pred = regr.predict(X_test)

# build table of actual vs. predicted values
accuracy_df = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
accuracy_df

,Actual,Predicted
29,534,469.391989
4,410,545.645464
26,577,589.668394
30,571,569.730413
32,577,649.774809
37,704,646.631164
34,487,511.608148
40,587,672.475177
7,467,502.074782
10,580,501.270734


Now let's calculate the MAE, MSE and RMSE of the observations to assess our model overall.

In [21]:
# load libraries
from sklearn import metrics

# calculate error scores, outcome mean and proportion
mae = round(metrics.mean_absolute_error(y_test, y_pred),2)
mse = round(metrics.mean_squared_error(y_test, y_pred),2)
rmse = np.sqrt(mse)
mean_cons = car_df['Consumption of petrol (millions of gallons)'].mean()
# means = pd.DataFrame(X.mean(), X.columns, columns=['Mean'])
accuracy = pd.DataFrame([mae, mse, rmse], ['MAE', 'MSE', 'RMSE'], columns=['Accuracy'])
prop = round(rmse/mean_cons,2)

# show results
print(accuracy)
print('Proportion: ', prop)

         Accuracy
MAE     56.820000
MSE   4666.340000
RMSE    68.310614
Proportion:  0.12


The above tells us that the RMSE for our model is 68 million gallons of petrol, this is about 12% of the mean of our petrol consumption variable, meaning that this is an OK predictor of values.

There are a number of factors which may affect the accuracy of our model:
* The features used aren't the best predictors of petrol consumption (might need more data/features)
* Our sample size isn't very big
* We may have assumed the relationship between the variables is linear when another model may be better (e.g. logistic or polynomial regression)